# Übung 4.1 : Das Traglinienverfahren und die Prandtl'sche Tragflügeltheorie

<div style="text-align: justify"> </p>Die Übung 4 ist die erste von insgesamt fünf computer-basierten Übungen und soll einen ersten Einblick in aerodynamische Optimierungsaufgaben geben und wie sie mit Algorithmen gelöst werden können. </p>

Die **Prandtl'sche Traglinientheorie (engl. "lifting line theory")** besteht seit der Zeit des ersten Weltkrieges und findet heutzutage immernoch Anwendung. Darauf basierend wird das numerische Traglinienverfahren als analytische Methode eingesetzt, um im Flügel-Vorentwurf Vorhersagen über Effizienz und Verluste von Tragflügeln endlicher Streckung zu treffen.

#### Was liefert die Traglinientheorie?
- Auftriebsverteilung entlang der Spannweite
- Auftriebskraft 
- Induzierter Widerstand (Effizienz)

#### Was sind die Einschränkungen?
- gilt nur für große Streckungen
- Pfeilung kann nicht berücksichtigt werden 
- gilt nur für kleine Anstellwinkel

Die ausführliche Theorie zum Verfahren findet ihr in den Skripten begleitend zur Vorlesung. Nachfolgend soll nun exemplarisch gezeigt werden, wie ein solches Traglinienverfahren in Python implementiert werden kann
 </div>

In [226]:
# Importieren von Biblioteken
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider, fixed

## Aufgabe 4.1 a)
Untersuchen Sie am Beispiel des Rechteckflügels mit $l_i=1$ und $\Lambda = 10$ das Konvergenzverhalten der aerodynamischen Beiwerte ($C_W$, $C_A$) und des Oswald Faktors hinsichtlich der Abhängigkeit von der Anzahl der Elementarflügel (räumliche Diskretisierung).
Entscheiden Sie sich für eine Diskretisierung für alle folgenden Aufgabenteil und begründen Sie diese Entscheidung.

### 4.1.1 Visualisierung: Doppeltrapezflügel

In [227]:
# TODO elliptische Planform gleicher Streckung hinterlegen
def doppeltrapezfluegel(l_a, l_k, AR, l_i=1):
    b = AR * (l_k*0.75 + 0.5*(l_i-l_k)*0.75+l_a*(1-0.75)+0.5*(l_k-l_a)*0.25)
    s = b / 2
    y_lk = 0.75 * s 

    # Definition der Punkte für eine Seite
    points = np.array([
        [0, l_i/2],              
        [y_lk, l_k/2],    
        [s, l_a/2],         
        [s, -l_a/2],         
        [y_lk, -l_k/2],         
        [0, -l_i/2],
        [-y_lk, -l_k/2],  
        [-s, -l_a/2],    
        [-s, l_a/2] ,         
        [-y_lk, l_k/2],
        [0, l_i/2]
    ])
    
    return points


def plot_doppeltrapezfluegel(l_a, l_k, AR):
    points = doppeltrapezfluegel(l_a, l_k, AR)

    plt.figure(figsize=(10, 6))
    plt.plot(points[:,0], points[:, 1], '-', color="black")
    plt.xlabel('Spannweite [m]')
    plt.ylabel('Flügeltiefe [m]')
    plt.title('Doppeltrapezflügel')
    plt.axis('equal')
    plt.show()

interact(plot_doppeltrapezfluegel, 
         l_a=FloatSlider(value=0.5, min=0, max=1.0, step=0.1, description='l_a'), 
         l_k=FloatSlider(value=1.0, min=0, max=1.0, step=0.1, description='l_k'), 
         AR=FloatSlider(value=10, min=5, max=20, step=1, description=r'$\Lambda$'))

interactive(children=(FloatSlider(value=0.5, description='l_a', max=1.0), FloatSlider(value=1.0, description='…

<function __main__.plot_doppeltrapezfluegel(l_a, l_k, AR)>

### 4.1.2 Geometrische Parameter und Randbedinungen

Für die anschließenden Berechnung

In [228]:
# Konstant
alpha = 1           # Anstellwinel [rad]
U_inf = 10          # Anströmgeschwindigkeit [m/s]
rho = 1.225         # Dichte [kg/m^3]
l_i = 1             # Flügeltiefe Wurzel [m]

# Variabel
l_a = 0.5           # Flügeltiefe außen [m]
l_k = 1             # Flügeltiefe kink [m]
AR = 10             # Streckung/ Aspect Ratio

#### Berechnung weiterer geometrischer Parameter

In [229]:
b = AR * (l_k*0.75 + 0.5*(l_i-l_k)*0.75+l_a*(1-0.75)+0.5*(l_k-l_a)*0.25)    # Spannweite
s = b / 2                                                                   # Halbspannweite
y_lk = 0.75 * s                                                             # Kink-Position bei 75% der Halbspannweite
S = b**2 / AR                                                               # Flügelfläche

# Ausgabe
print("Spannweite:                  ", "%.2f" % b, "m")
print("Halbspannweite:              ", "%.2f" % s, "m")
print("Kink-Position:               ", "%.2f" % y_lk, "m")
print("Flügelfläche:                ", "%.2f" % S, "m^2")

Spannweite:                   9.38 m
Halbspannweite:               4.69 m
Kink-Position:                3.52 m
Flügelfläche:                 8.79 m^2


#### Vergleich mit elliptischer Planform

In [230]:
h_ell = l_i/2 * (np.sqrt(1 - (y_lk / s)))                                   # ???                   
S_ell = np.pi * l_i * 0.5 * s                                               # entsprechende Fläche
AR_ell = b**2 / S_ell                                                       # entsprechende Streckung

# Ausgabe
print("???:                         ", "%.2f" % h_ell, "m")
print("Flügelfläche:                ", "%.2f" % S_ell, "m^2")
print("Streckung:                   ", "%.2f" % AR_ell)

???:                          0.25 m
Flügelfläche:                 7.36 m^2
Streckung:                    11.94


### 4.1.3 Diskretisierung der Spannweite

In [231]:
def plot_doppeltrapezfluegel(l_a, l_k, AR,N):
    points = doppeltrapezfluegel(l_a, l_k, AR)
    
    # Diskretisierung der Halbspannweite
    y_values = np.linspace(-s, s, N+1)
    
    # Tiefe des Flügels an verschiedenen y-Positionen berechnen
    def tiefe(y):
        y = abs(y)
        if y <= y_lk:
            return 1 + (l_k - 1) * (y / y_lk)
        else:
            return l_k + (l_a - l_k) * ((y - y_lk) / (s - y_lk))

    chord_values = [tiefe(y) for y in y_values]
    
    # Plotten des Flügels
    plt.figure(figsize=(10, 6))
    plt.plot(points[:,0], points[:, 1], '-', color="black")

    # Diskretisierte Punkte entlang der Spannweite
    for i in range(N):
        #plt.plot([y_values[i], y_values[i+1]], [chord_values[i]/2, chord_values[i+1]/2], 'r--')
        # plt.plot([y_values[i], y_values[i+1]], [-chord_values[i]/2, -chord_values[i+1]/2], 'r--')
        plt.plot([y_values[i], y_values[i]], [-chord_values[i]/2, chord_values[i]/2], 'g-')
    
    plt.xlabel('Spannweite [m]')
    plt.ylabel('Flügeltiefe [m]')
    plt.title('Diskretisierung Doppeltrapezflügel')
    plt.axis('equal')
    plt.grid(True)
    plt.show()

# Gegebene Parameter
l_a = 0.5
l_k = 1.0
AR = 10

# TODO es entspricht nicht dem code dass die elementarflügel von der flügelform abweichen
interact(plot_doppeltrapezfluegel, 
         l_a=fixed(l_a), 
         l_k=fixed(l_k), 
         AR=fixed(AR), 
         N=IntSlider(value=30, min=1, max=100, step=1, description='N'))

interactive(children=(IntSlider(value=30, description='N', min=1), Output()), _dom_classes=('widget-interact',…

<function __main__.plot_doppeltrapezfluegel(l_a, l_k, AR, N)>

### 4.1.4 Berechnung der Hufeisenwirbel-Koordinaten

Zur Bestimmung der $N$ (Variable n) unbekannten Wirbelstärken $\Gamma$ wird die kinematische Strömungsbedingung in den $N$ (Variable $m$) Aufpunkten erfüllt, und man hat ein lineares Gleichungssystem zu lösen. Dafür werden die Koordinaten des tragenden Wirbels entlang der örtlichen $l_p/4$-Linie (indizes a, b) am Rand jedes Elementarflügels sowie die der Aufpunkte im $3l_p/4$–Punkt des Mittelschnittes jedes Elementarflügels (index c) bestimmt.

In [232]:
N = 2                         # Anzahl der Elementarflügel
delta_b = b/N                   # Breite eines Elementarflügels

In [233]:
# Spannweitige Positionen der Punkte A, B und Aufpunkt C
y_a = np.array([-s +  j    * delta_b for j in range(N)])
y_b = np.array([-s + (j+1) * delta_b for j in range(N)])
y_c = np.array([-s + delta_b/2 + i*delta_b for i in range(N)])

# Tiefenposition der Punkte A und B und Aufpunkt C
def x_position(y, l_i, l_k, l_a, y_lk, aufpunkt=False):
    tiefe = 0.75 if aufpunkt else 0.25
    if abs(y) <= y_lk:
        return (abs(y)/y_lk * 0.5*(l_i-l_k) + tiefe*(l_i - abs(y)/y_lk * (l_i-l_k)))
    else:
        return (0.5*(l_i-l_k) + (abs(y)-y_lk)/(s-y_lk) * 0.5*(l_k-l_a) + tiefe*(l_k - (abs(y)-y_lk)/(s-y_lk)*(l_k-l_a)))
    
x_a = np.array([x_position(y_i, l_i, l_k, l_a, y_lk) for y_i in y_a])
x_b = np.array([x_position(y_i, l_i, l_k, l_a, y_lk) for y_i in y_b])
x_c = np.array([x_position(y_i, l_i, l_k, l_a, y_lk, True) for y_i in y_c])

# Spannweitige Positionen der bis ins Unendliche reichenden nichttragenden Wirbel
y_aoo = np.array([-s +  j    * delta_b for j in range(N)])
y_boo = np.array([-s + (j+1) * delta_b for j in range(N)])

# Tiefenposition der bis ins Unendliche reichenden nichttragenden Wirbel
x_aoo = x_a + 100*s
x_boo = x_b + 100*s

### 4.1.5 Berechnung der Koeffizientenmatrix

In [234]:
# initialisieren der Matrizen
a_AB = np.zeros((x_a.size, x_a.size))
a_Aoo = np.zeros((x_a.size, x_a.size))
a_Boo = np.zeros((x_a.size, x_a.size))

for j in range(N):
    for i in range(N):
        a_AB[i, j] = 1 / (4*np.pi) * (1/((x_c[i] - x_a[j])*(y_c[i] - y_b[j]) - (y_c[i] - y_a[j])*(x_c[i] - x_b[j]))) * (((x_b[j] - x_a[j])*(x_c[i] - x_a[j]) + (y_b[j] - y_a[j])*(y_c[i] - y_a[j]))/ np.sqrt((x_c[i] - x_a[j])**2 + (y_c[i] - y_a[j])**2) - ((x_b[j] - x_a[j])*(x_c[i] - x_b[j]) + (y_b[j] - y_a[j])*(y_c[i] - y_b[j]))/ np.sqrt((x_c[i] - x_b[j])**2 + (y_c[i] - y_b[j])**2))

for j in range(N):
    for i in range(N):
        a_Aoo[i, j] = 1 / (4*np.pi) * 1/(y_a[j] - y_c[i]) * ((x_c[i] - x_a[j])/ (np.sqrt((x_c[i] - x_a[j])**2 + (y_c[i] - y_a[j])**2)) + 1)

for j in range(N):
    for i in range(N):
        a_Boo[i, j] = 1 / (4*np.pi) * 1/(y_b[j] - y_c[i]) * ((x_c[i] - x_b[j])/ (np.sqrt((x_c[i] - x_b[j])**2 + (y_c[i] - y_b[j])**2)) + 1)

print(a_AB)
print(a_Aoo)
print(a_Boo)

[[-0.35774104 -0.00349879]
 [-0.00349879 -0.35774104]]
[[-0.03931731  0.04103697]
 [-0.01192044 -0.04103697]]
[[ 0.04103697  0.01192044]
 [-0.04103697  0.03931731]]


In [235]:
a = a_AB + a_Aoo - a_Boo                # Koeffizientenmatrix
a_ind = a_Aoo - a_Boo                   # Koeffizientenmatrix der induzierenden Wirbel
alpha_g = np.full((x_a.size), alpha)    # Anstellwinkelvektor

# die Wirbelstärken werden aus Inverser der Koeffizientenmatrix, Anstellwinkelvektor und Anströmgeschwindigkeit berechnet
Gamma = np.linalg.inv(a) * alpha_g * (- U_inf)

### 4.1.6 Berechnung von Kräften und Beiwerten

In [236]:
# Berechnung des Auftriebs nach Kutta-Joukowsky
A = np.sum(rho * delta_b * U_inf * Gamma)

# Berechnung des induzierten Widerstands
w_m = a_ind * Gamma
W_ind = np.sum(-rho *w_m * Gamma * delta_b)

print("Auftrieb:                    ", "%.2f" % A, "N")
print("Induzierter Widerstand:      ", "%.2f" % W_ind, "N")

Auftrieb:                     2784.24 N
Induzierter Widerstand:       483.52 N


In [237]:
# Auftriebsbeiwert
C_A = 2*A / (rho * U_inf**2 * S)

# Für alpha= 1 [rad] entspricht C_A dem Auftriebsanstieg wegen C_A = C_A_alpha * alpha
# Als Vergleichswert dient die Näherungslösung
C_A_alpha = 2*np.pi * AR / (2+AR)

# Widerstandsbeiwert (induziert)
C_W_ind = 2*W_ind / (rho * U_inf**2 * S)

print("Auftriebsbeiwert:            ", "%.2f" % C_A)
print("Auftriebsanstieg:            ", "%.2f" % C_A_alpha)
print("Widerstandsbeiwert:          ", "%.2f" % C_W_ind)

Auftriebsbeiwert:             5.17
Auftriebsanstieg:             5.24
Widerstandsbeiwert:           0.90


In [238]:
# Induzierter Widerstand der elliptischen Zirkulationsverteilung
C_W_ind_ell = C_A**2 / AR / np.pi

# Oswald Faktor
Oswald = C_W_ind_ell / C_W_ind

print("Widerstandsbeiwert (Ellipse):", "%.2f" % C_W_ind_ell)
print("Oswald-Faktor:               ", "%.2f" % Oswald)

Widerstandsbeiwert (Ellipse): 0.85
Oswald-Faktor:                0.95
